<a href="https://colab.research.google.com/github/tomasonjo/blogs/blob/master/neo4jdocs/Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install selenium==4.7.2 transformers sentence-transformers

  Using cached torch-1.10.2-cp38-cp38-manylinux1_x86_64.whl (881.9 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0
    Uninstalling torch-1.11.0:
      Successfully uninstalled torch-1.11.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
zero-shot-re 0.0.4 requires torch==1.9.0, but you have torch 1.10.2 which is incompatible.
zero-shot-re 0.0.4 requires transformers==4.9.2, but you have transformers 4.17.0 which is incompatible.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.10.2 which is incompatible.
crosslingual-coreference 0.2.1 requires click<8.1.0, but you have click 8.1.3 which is incompatible.
classy-core 0.2.1 requires datasets==1.12.1, but you have datasets 2.1.0 which is incompatible.
classy-core 0.2.1 requires scikit-learn==0.24.2, but you have scikit-learn 1.1.1 which is incompatible.
classy-core 0.2.1

In [2]:
!apt-get update
!apt install chromium-chromedriver
!apt install -y xvfb

!pip install undetected-chromedriver
!pip install PyVirtualDisplay

Reading package lists... Done
E: Could not open lock file /var/lib/apt/lists/lock - open (13: Permission denied)
E: Unable to lock directory /var/lib/apt/lists/
W: Problem unlinking the file /var/cache/apt/pkgcache.bin - RemoveCaches (13: Permission denied)
W: Problem unlinking the file /var/cache/apt/srcpkgcache.bin - RemoveCaches (13: Permission denied)
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
     |████████████████████████████████| 58 kB 1.9 MB/s eta 0:00:01
  Created wheel for undetected-chromedriver: filename=undetected_chromedriver-3.1.7-py3-none-any.whl size=45442 sha256=7c17d6d2f7ba130a69098adc97ead83801e80a09a0f4714cc727e525a5895cef
  Stored in directory: /home/tomaz/.cac

In [3]:
!zip -j /content/chromedriver_linux64.zip /usr/bin/chromedriver
# replace python3.8 with your own version of python in case it's not the same
patcher_src = (
    "/usr/local/lib/python3.8/dist-packages/undetected_chromedriver/patcher.py"
)
with open(patcher_src, "r") as f:
    contents = f.read()
    contents = contents.replace(
        "return urlretrieve(u)[0]",
        "return urlretrieve('file:///content/chromedriver_linux64.zip',"
        "filename='/tmp/chromedriver_linux64.zip')[0]",
    )
with open(patcher_src, "w") as f:
    f.write(contents)

  adding: chromedriver (deflated 51%)


In [4]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [5]:
import undetected_chromedriver.v2 as uc
from pyvirtualdisplay import Display

display = Display(visible=0, size=(800, 600))
display.start()

options = uc.ChromeOptions()
options.add_argument("--no-sandbox")
wd = uc.Chrome(options=options)

In [6]:
def extract_text_by_class(c):
    global wd
    try:
        content = wd.find_element(By.CLASS_NAME, c)
        return content.text
    except:
        return ""


def extract_links_by_xpath(xpath):
    global wd
    links = set()
    try:
        a_elems = wd.find_elements(By.XPATH, xpath)
        for elem in a_elems:
            link = elem.get_attribute("href")
            if link == "javascript:void(0)":
                continue
            # Remove links to images and various files
            if (
                link.endswith(".png")
                or link.endswith(".json")
                or link.endswith(".txt")
                or link.endswith(".svg")
                or link.endswith(".ipynb")
                or link.endswith(".jpg")
                or link.endswith(".pdf")
                or link.endswith(".mp4")
            ):
                continue
            # Remove anchors
            link = link.split("#")[0]
            # Remove parameters
            link = link.split("?")[0]
            # Remove trailing forward slash
            link = link.rstrip("/")
            links.add(link)
        return list(links)
    except:
        return []

In [7]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoModel
from transformers import pipeline
import torch

tokenizer = AutoTokenizer.from_pretrained("yanekyuk/bert-uncased-keyword-extractor")
model = AutoModelForTokenClassification.from_pretrained(
    "yanekyuk/bert-uncased-keyword-extractor"
)

nlp = pipeline("ner", model=model, tokenizer=tokenizer)


def extract_keywords(text):
    """
    Extract keywords and construct them back from tokens
    """
    result = list()
    keyword = ""
    ner_results = nlp(text)
    a = [x["word"] for x in ner_results]
    for token in a:
        if token.startswith("##"):
            keyword += token[2:]
        else:
            if keyword:
                result.append(keyword)
            keyword = token
    return result

Downloading:   0%|          | 0.00/321 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/695k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/809 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/415M [00:00<?, ?B/s]

In [8]:
extract_keywords(
    """
Web APIs are a huge opportunity to access and integrate data from any sources with your graph. Most of them provide the data in JSON format.

The Load JSON procedures retrieve data from URLs or maps and turn it into map value(s) for Cypher to consume. Cypher has support for deconstructing nested documents with dot syntax, slices, UNWIND etc. so it is easy to turn nested data into graphs.

Sources with multiple JSON objects (JSONL,JSON Lines) in a stream, like the streaming Twitter format or the Yelp Kaggle dataset, are also supported,
"""
)

['graph', 'json', 'cypher', 'syntax', 'twitter', 'yelp', 'kaggle']

In [9]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")


def generate_embeddings(text):
    embeddings = model.encode(text)
    return embeddings

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [10]:
generate_embeddings(
    """
Web APIs are a huge opportunity to access and integrate data from any sources with your graph. Most of them provide the data in JSON format.

The Load JSON procedures retrieve data from URLs or maps and turn it into map value(s) for Cypher to consume. Cypher has support for deconstructing nested documents with dot syntax, slices, UNWIND etc. so it is easy to turn nested data into graphs.

Sources with multiple JSON objects (JSONL,JSON Lines) in a stream, like the streaming Twitter format or the Yelp Kaggle dataset, are also supported,
"""
)

array([-6.91139922e-02,  5.22450246e-02, -7.20178010e-03, -1.53747611e-02,
        3.90604814e-03, -5.04245870e-02, -1.06758319e-01,  6.22570850e-02,
        3.25474143e-03,  1.30456192e-02,  7.43217766e-03,  2.43864255e-03,
        7.32425004e-02,  6.01712838e-02,  1.34542912e-01,  3.66639905e-02,
       -7.08860904e-03,  3.01807802e-02,  2.62599122e-02, -1.47831604e-01,
        1.14754057e-02,  1.68467127e-03, -4.65278402e-02, -3.76555063e-02,
        4.88043539e-02,  8.55944529e-02, -3.02659832e-02,  5.21277003e-02,
        4.56753932e-02,  6.37385473e-02, -2.97765397e-02, -1.06713418e-02,
       -5.80438487e-02,  5.81364371e-02, -9.82052982e-02,  9.66787804e-03,
        3.53001878e-02,  9.53013543e-03,  1.33904722e-02,  5.26607819e-02,
        4.21146378e-02,  1.21976286e-02, -4.74480391e-02,  1.82718318e-02,
       -2.74320948e-03, -1.94643103e-02, -8.98622945e-02,  4.01606634e-02,
       -5.03845848e-02,  5.59670553e-02, -1.13629214e-01, -4.88040783e-02,
        9.09033604e-03,  

In [11]:
from selenium.webdriver.common.by import By

entry_url = "https://neo4j.com/docs"
data = dict()
visit_list = [entry_url]
already_visited = []

while visit_list:
    # Visit the URL
    current_url = visit_list.pop()
    if current_url in already_visited:
        continue
    print(current_url)
    try:
        wd.get(current_url)
    except:
        print(f"Couldn't open {current_url}")
    # Extract text from the content div
    text = extract_text_by_class("content")
    # If nothing is found, try article div
    if not text:
        text = extract_text_by_class("article")

    # Generate paragraph embedding & extract keywords
    if text:
        embeddings = generate_embeddings(text)
        keywords = extract_keywords(text)
    else:
        embeddings = []
        keywords = []

    # Extract links from the content div
    links = extract_links_by_xpath("//div[@class='content']//a[@href]")
    # If nothing is found, try article div
    if not links:
        links = extract_links_by_xpath("//article[@class='article']//a[@href]")

    if not links and not text:
        print(f"Couldn't retrieve the data from {current_url}")
    # Store page information
    data[current_url] = {
        "links": links,
        "text": text,
        "embeddings": embeddings,
        "keywords": keywords,
    }
    # Crawling information
    already_visited.append(current_url)
    # Don't leave neo4j.com while crawling
    visit_list.extend(
        [l for l in list(links) if "neo4j.com" in l and not l in already_visited]
    )

https://neo4j.com/docs
https://neo4j.com/docs/getting-started
https://community.neo4j.com
Couldn't retrieve the data from https://community.neo4j.com
https://neo4j.com/docs/getting-started/current/graph-visualization/graph-visualization
https://neo4j.com/graphconnect-2018/session/neo4j-spatial-mapping
Couldn't retrieve the data from https://neo4j.com/graphconnect-2018/session/neo4j-spatial-mapping
https://neo4j.com/docs/getting-started/5/integration-tools/integration
https://neo4j.com/docs/getting-started/5/graph-visualization/graph-visualization
https://neo4j.com/bi-connector
Couldn't retrieve the data from https://neo4j.com/bi-connector
https://neo4j.com/docs/getting-started/5/graph-visualization/graph-visualization-tools
https://neo4j.com/aura
Couldn't retrieve the data from https://neo4j.com/aura
https://neo4j.com/blog/evaluating-investor-performance-using-neo4j-graphxr-and-ml
Couldn't retrieve the data from https://neo4j.com/blog/evaluating-investor-performance-using-neo4j-graphxr

https://neo4j.com/docs/operations-manual/5/tools/neo4j-admin/neo4j-admin-memrec
https://neo4j.com/docs/operations-manual/5/docker/operations
https://neo4j.com/docs/bloom-user-guide/current
https://neo4j.com/docs/bloom-user-guide/current/about-bloom
https://neo4j.com/docs/bloom-user-guide/2.5/bloom-perspectives/perspective-storage
https://neo4j.com/docs/operations-manual/current/manage-databases/introduction
https://neo4j.com/docs/operations-manual/5/authentication-authorization/access-control
https://neo4j.com/docs/cypher-manual/5/access-control
https://neo4j.com/docs/cypher-manual/5/access-control/manage-users
https://neo4j.com/docs/operations-manual/5.1/configuration/set-initial-password
https://neo4j.com/docs/operations-manual/5/configuration/connectors
https://neo4j.com/docs/operations-manual/5/security/ssl-framework
https://neo4j.com/docs/operations-manual/5/configuration/neo4j-conf
https://neo4j.com/docs/operations-manual/5/performance/gc-tuning
https://neo4j.com/docs/operations-

https://neo4j.com/docs/operations-manual/5/reference
https://neo4j.com/docs/operations-manual/5/tools/cypher-shell
https://neo4j.com/docs/operations-manual/5/tools/neo4j-admin/migrate-configuration
https://neo4j.com/docs/operations-manual/5/tools/neo4j-admin/migrate-database
https://neo4j.com/docs/upgrade-migration-guide/current/version-5/migration/planning
https://neo4j.com/docs/upgrade-migration-guide/current/version-5/migration
https://neo4j.com/docs/upgrade-migration-guide/current/version-5/upgrade-minor/new-server-rolling
https://neo4j.com/docs/upgrade-migration-guide/current/version-5/upgrade-minor
https://neo4j.com/docs/upgrade-migration-guide/current/version-5
https://neo4j.com/docs/upgrade-migration-guide/current/version-4
https://neo4j.com/docs/upgrade-migration-guide/current/version-4/planning
https://neo4j.com/docs/upgrade-migration-guide/current/version-4/migration/drivers
https://neo4j.com/docs/upgrade-migration-guide/current/version-4/migration/drivers/go-driver
https://

https://neo4j.com/labs/apoc/4.2/overview/apoc.util/apoc.util.compress
https://neo4j.com/labs/apoc/4.2/overview/apoc.util/apoc.util.decompress
https://neo4j.com/labs/apoc/4.2/overview/apoc.util/apoc.util.md5
https://neo4j.com/labs/apoc/4.2/overview/apoc.util/apoc.util.sha1
https://neo4j.com/labs/apoc/4.2/overview/apoc.util/apoc.util.sha256
https://neo4j.com/labs/apoc/4.2/overview/apoc.util/apoc.util.sha384
https://neo4j.com/labs/apoc/4.2/overview/apoc.util/apoc.util.sha512
https://neo4j.com/labs/apoc/4.2/overview/apoc.util/apoc.util.validatePredicate
https://neo4j.com/labs/apoc/4.2/overview/apoc.uuid
https://neo4j.com/labs/apoc/4.2/overview/apoc.uuid/apoc.uuid.removeAll
https://neo4j.com/labs/apoc/4.2/overview/apoc.warmup
https://neo4j.com/labs/apoc/4.2/overview/apoc.warmup/apoc.warmup.run
https://neo4j.com/labs/apoc/4.2/overview/apoc.xml
https://neo4j.com/labs/apoc/4.2/overview/apoc.xml/apoc.xml.parse
https://neo4j.com/labs/apoc/4.2/overview/apoc.xml/apoc.xml.import
https://neo4j.com/l

https://neo4j.com/docs/operations-manual/5/kubernetes/quickstart-standalone
https://neo4j.com/docs/operations-manual/5/kubernetes/quickstart-standalone/install-neo4j
https://neo4j.com/docs/operations-manual/5/kubernetes/quickstart-standalone/create-value-file
https://neo4j.com/docs/operations-manual/5/kubernetes/quickstart-standalone/prerequisites
https://neo4j.com/docs/operations-manual/5/installation/requirements
https://neo4j.com/docs/operations-manual/5/installation/neo4j-desktop
https://neo4j.com/docs/operations-manual/5/installation/neo4j-browser
https://neo4j.com/docs/browser-manual/current
https://neo4j.com/docs/browser-manual/current/deployment-modes
https://neo4j.com/docs/browser-manual/4.4/about-browser
https://neo4j.com/docs/browser-manual/4.4/deployment-modes
https://neo4j.com/docs/browser-manual/4.4/deployment-modes/neo4j-server
https://neo4j.com/docs/browser-manual/4.4/deployment-modes/neo4j-desktop
https://neo4j.com/docs/browser-manual/4.4/deployment-modes/web-applicati

...

https://neo4j.com/docs/java-manual/4.4/session-api
https://neo4j.com/docs/java-manual/4.4/cypher-workflow
https://neo4j.com/docs/java-manual/4.4/get-started
https://neo4j.com/docs/api/java-driver/4.4
Couldn't retrieve the data from https://neo4j.com/docs/api/java-driver/4.4
https://neo4j.com/docs/api/java-driver/4.4/org/neo4j/driver/Value.html
Couldn't retrieve the data from https://neo4j.com/docs/api/java-driver/4.4/org/neo4j/driver/Value.html
https://neo4j.com/docs/python-manual/4.4
https://neo4j.com/docs/python-manual/4.4/terminology
https://neo4j.com/docs/python-manual/4.4/session-api
https://neo4j.com/docs/python-manual/4.4/cypher-workflow
https://neo4j.com/docs/python-manual/4.4/client-applications
https://neo4j.com/docs/api/python-driver/4.4/api.html
Couldn't retrieve the data from https://neo4j.com/docs/api/python-driver/4.4/api.html
https://neo4j.com/docs/python-manual/4.4/get-started
https://neo4j.com/docs/api/python-driver/4.4
Couldn't retrieve the data from https://neo4j.co

https://neo4j.com/docs/operations-manual/4.2/tools
https://neo4j.com/docs/java-reference/4.2/index
Couldn't retrieve the data from https://neo4j.com/docs/java-reference/4.2/index
https://neo4j.com/docs/cypher-manual/4.2/index
Couldn't retrieve the data from https://neo4j.com/docs/cypher-manual/4.2/index
https://neo4j.com/docs/cypher-manual/4.2/access-control
Couldn't retrieve the data from https://neo4j.com/docs/cypher-manual/4.2/access-control
https://neo4j.com/docs/cypher-manual/4.2/administration/security/built-in-roles
https://neo4j.com/docs/cypher-manual/4.2/administration/security/limitations
https://neo4j.com/docs/cypher-manual/4.2/administration/security/introduction
https://neo4j.com/docs/java-reference/4.2/javadocs/org/neo4j/procedure/UserAggregationFunction.html
Couldn't retrieve the data from https://neo4j.com/docs/java-reference/4.2/javadocs/org/neo4j/procedure/UserAggregationFunction.html
https://neo4j.com/docs/operations-manual/current/tools/neo4j-admin/neo4j-admin-store

Couldn't retrieve the data from https://neo4j.com/docs/java-reference/4.4/javadocs/org/neo4j/procedure/UserFunction.html
https://neo4j.com/docs/graph-data-science-client/1.5/model-object
https://neo4j.com/docs/graph-data-science-client/1.5/known-limitations
https://neo4j.com/docs/graph-data-science-client/1.5/tutorials/load-data-via-graph-construction
https://neo4j.com/docs/graph-data-science/current/management-ops/create-cypher-db
https://neo4j.com/docs/graph-data-science-client/1.5/graph-object
https://neo4j.com/docs/graph-data-science/current/graph-project-apache-arrow
https://neo4j.com/docs/graph-data-science/current/graph-catalog-relationship-ops
https://neo4j.com/docs/graph-data-science/current/management-ops/administration
https://neo4j.com/docs/graph-data-science-client/1.5/algorithms
https://neo4j.com/docs/graph-data-science-client/1.5/pipelines
https://neo4j.com/docs/graph-data-science/current/machine-learning/linkprediction-pipelines/config
https://neo4j.com/docs/graph-data-

https://neo4j.com/docs/java-reference/3.5/javadocs/org/neo4j/graphdb/factory/HighlyAvailableGraphDatabaseFactory.html
Couldn't retrieve the data from https://neo4j.com/docs/java-reference/3.5/javadocs/org/neo4j/graphdb/factory/HighlyAvailableGraphDatabaseFactory.html
https://neo4j.com/docs/java-reference/3.5/javadocs/org/neo4j/graphdb/factory/EnterpriseGraphDatabaseFactory.html
Couldn't retrieve the data from https://neo4j.com/docs/java-reference/3.5/javadocs/org/neo4j/graphdb/factory/EnterpriseGraphDatabaseFactory.html
https://neo4j.com/docs/java-reference/3.5/javadocs/org/neo4j/graphdb/ResourceIterator.html
Couldn't retrieve the data from https://neo4j.com/docs/java-reference/3.5/javadocs/org/neo4j/graphdb/ResourceIterator.html
https://neo4j.com/docs/java-reference/3.5/javadocs/org/neo4j/logging/LogProvider.html
Couldn't retrieve the data from https://neo4j.com/docs/java-reference/3.5/javadocs/org/neo4j/logging/LogProvider.html
https://neo4j.com/docs/java-reference/3.5/javadocs/org/n

https://neo4j.com/labs/apoc/4.2/overview/apoc.do/apoc.do.when
https://neo4j.com/labs/apoc/4.2/overview/apoc.dv
https://neo4j.com/labs/apoc/4.2/overview/apoc.dv/apoc.dv.catalog.remove
https://neo4j.com/labs/apoc/4.2/overview/apoc.dv/apoc.dv.catalog.list
https://neo4j.com/labs/apoc/4.2/overview/apoc.dv/apoc.dv.catalog.add
https://neo4j.com/labs/apoc/4.2/overview/apoc.dv/apoc.dv.query
https://neo4j.com/labs/apoc/4.2/overview/apoc.dv/apoc.dv.queryAndLink
https://neo4j.com/labs/apoc/4.2/overview/apoc.es
https://neo4j.com/labs/apoc/4.2/overview/apoc.es/apoc.es.get
https://neo4j.com/labs/apoc/4.2/overview/apoc.es/apoc.es.getRaw
https://neo4j.com/labs/apoc/4.2/overview/apoc.es/apoc.es.postRaw
https://neo4j.com/labs/apoc/4.2/overview/apoc/apoc.case
https://neo4j.com/labs/apoc/4.2/overview/apoc/apoc.help
https://neo4j.com/labs/apoc/4.2/overview/apoc/apoc.when
https://neo4j.com/labs/apoc/4.2/overview/apoc/apoc.version
https://neo4j.com/labs/apoc/4.2/overview/apoc.agg
https://neo4j.com/labs/apoc/4

In [19]:
len(data.keys())

10319

In [27]:
data["https://neo4j.com/labs/kafka"]

{'links': ['https://neo4j.com/docs/license',
  'https://neo4j.com/docs/kafka/neo4j-cluster-connect',
  'https://neo4j.com/docs/kafka/quickstart-connect',
  'https://neo4j.com/docs/kafka/architecture',
  'https://neo4j.com/docs/kafka/kafka-connect',
  'https://neo4j.com/docs/kafka-streams',
  'https://neo4j.com/docs/kafka/overview'],
 'text': 'Kafka Connect Neo4j Connector User Guide\nThe documentation on the deprecated Neo4j Streams plugin and the previous version of the Kafka Connect Neo4j Connector can be found here.\nThe guide covers the following areas:\nProject overview\nQuick start with Kafka Connect Neo4j Connector\nHow to install and configure Kafka Connect Neo4j Connector\nUsage with a Neo4j cluster\nArchitectural Guidance\n© 2022 License: Creative Commons 4.0\nProject overview\nWas this page helpful?',
 'embeddings': [-0.02007158473134041,
  -0.0022500907070934772,
  -0.10914726555347443,
  -0.013005321845412254,
  -0.011719848960638046,
  0.058137692511081696,
  -0.097378686

In [18]:
for key in list(data.keys()):
    if "mailto" in key:
        del data[key]

In [26]:
for key in data:
    data[key]["keywords"] = list(set(data[key]["keywords"]))

In [28]:
import json

j = json.dumps(data)
# open file for writing, "w"
f = open("neo4j_docs.json", "w")

# write json object to file
f.write(j)

# close file
f.close()